In [12]:
import pandas as pd
from bestStations import getDelta 
from ipywidgets import widgets
from IPython.display import display
import folium

In [13]:
# Parse neighborhoods file, sort it alphabetically
neighbourhoods = pd.read_csv('./neighbourhoods.csv')
neighbourhoods = neighbourhoods.sort_values(['neighbourhood'])
# print(neighbourhoods);

# Prepare it to populate dropdown as "Midtown":11 style.. 
# User can selecft Midtown but selecting value will be integer 11
nhoods = {}
for i in range(len(neighbourhoods)):
    nhoods[neighbourhoods.iloc[i]['neighbourhood']] = neighbourhoods.iloc[i]['id']
# print(dict)


In [14]:
# A common layout object to be used by dropdowns, etc..
form_item_layout = widgets.Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between'
)

In [15]:
# Dropdown to select a nhood
wNhood = widgets.Dropdown(
    options = nhoods,
    description='Neighbourhood ',
    disabled=False,
    value=11,
    layout = form_item_layout
)
# display(wNhood)

In [16]:
# Dropdown to select a day
wDay = widgets.Dropdown(
    options = {"Monday":0, "Tuesday":1, "Wednesday":2, "Thursday":3, "Friday":4, "Saturday":5, "Sunday":6},
    description='Day ',
    disabled=False,
    layout = form_item_layout )
# display(wNhood)

In [17]:
# Slider to select an hour
wHour = widgets.IntSlider(max = 23, value=10, description="Hour ", 
                          layout = form_item_layout )

wHour.style.handle_color = 'lightgreen'
# display(wHour)

In [18]:
# Output console to print results
wOut = widgets.Output()
# display(wOut)

In [19]:
# Output to display a map
wMap = widgets.Output()

In [20]:
# Function to run when User presses the button
def getResults(sender):
    print("Finding: ", wNhood.value, wDay.value, wHour.value)
    
    # run the delta with user input 
    best = getDelta(wNhood.value, wDay.value, wHour.value, 10000)
    
    # Create NYC map, add top-k stations as markers
    NYC_COORDINATES = (40.760368, -73.985181)
    m = folium.Map(location = NYC_COORDINATES, zoom_start=13)
    top = 0;
    for station in best.itertuples():
        #  station[1] is delta as pop up
        delta = station[1]
        
        # Select color
        color = 'blue'
        if(top < 5):
            color = 'green'
            top += 1  
        if(delta < 0):
            color = 'red'
            
        # Add marker station[2] and station[3] are x, y values and 
        folium.Marker([ station[2], station[3]], popup=str(station[1]), icon=folium.Icon(color)).add_to(m)
    
    # Clear and print results
    wOut.clear_output()
    with wOut:
        print(best)
    
    # Clear and map results
    wMap.clear_output()
    with wMap:
        display(m)


In [21]:
wButton = widgets.Button(description="Find Stations!"
                        ,layout = widgets.Layout(width='150px', height='50px')) #form_item_layout)
# wButton.style.button_color = 'lightgreen'
wButton.button_style = "info"
wButton.on_click(getResults)
# display(wButton)

In [22]:
box_layout = widgets.Layout(
    display='flex',
    flex_flow='column',
    border='solid 2px',
    align_items='center', #flex-start, flex-end, center, baseline, stretch
    width='40%')

top_box = widgets.HBox([wNhood, wDay, wHour, wButton])
# layout = box_layout)

tab = widgets.Tab()
tab.children = [widgets.VBox([top_box, wMap, wOut])]
# widgets.HBox([left_box, wOut, wMap])]
tab.set_title(0, "Best Stations")
# tab.set_title(1, "XX")

tab

Finding:  11 0 10
